In [4]:
%reload_ext autoreload
%autoreload 2

import pnllib
import pandas as pd
import numpy as np
from positionslib import get_expiration_indexed_positions
pnllib.set_log_level(pnllib.ERROR)

In [5]:
NEEDED_COLUMNS=['Symbol', 'Quantity', 'Cost Basis', "PnL", "Margin", "Margin Requirement"]

filename="data/pos.csv"

dct=get_expiration_indexed_positions(filename, needed_columns=NEEDED_COLUMNS)

# Convert dictionary to DataFrame
rows = []
for date, df in dct.items():
    row = {
        'Date': date,
        'Cost Basis': int(-df['Cost Basis'].sum()),
        'Quantity': int(-df['Quantity'].sum()),
        'Premium': int(-df['Cost Basis'].sum().round()),  # Premium = Cost Basis
        'PnL': int(df['PnL'].sum()),
        'Margin': int(df['Margin'].sum())
    }
    rows.append(row)

# Create DataFrame indexed by date
weekly_df = pd.DataFrame(rows)
weekly_df['Date'] = pd.to_datetime(weekly_df['Date'])
weekly_df = weekly_df.set_index('Date')
weekly_df = weekly_df.sort_index()

print("Weekly DataFrame:")
print(weekly_df)
print("\n")

# Create monthly aggregated DataFrame
monthly_df = weekly_df.copy()
monthly_df['Month'] = monthly_df.index.to_period('M')
monthly_agg = monthly_df.groupby('Month').agg({
    'Cost Basis': 'sum',
    'Quantity': 'sum',
    'Premium': 'sum',
    'PnL': 'sum',
    'Margin': 'sum'
})

print("Monthly Aggregated DataFrame:")
print(monthly_agg)

Total margin 617716
Weekly DataFrame:
            Cost Basis  Quantity  Premium   PnL  Margin
Date                                                   
2025-11-21        7410       110     7410  2026  132269
2025-11-28        2229        38     2230  1198   53718
2025-12-05        3047        52     3048 -1737   60291
2025-12-12        3479        54     3479 -1991   74027
2025-12-19        5593        93     5594 -2688  103750
2025-12-26         511         6      511  -382    7503


Monthly Aggregated DataFrame:
         Cost Basis  Quantity  Premium   PnL  Margin
Month                                               
2025-11        9639       148     9640  3224  185987
2025-12       12630       205    12632 -6798  245571


In [6]:
d=dct["11/21/2025"]
d["gap"]=-d["Cost Basis"] - d.PnL
d.loc[d.PnL + d["Cost Basis"] < -25, ["Symbol", "Quantity", "Cost Basis", "PnL", "gap", "Margin"] ].reset_index()

,index,Symbol,Quantity,Cost Basis,PnL,gap,Margin
0,93,ANF 11/21/2025 67.00 P,-1,-384.34,294.33,90.01,1210.8
1,94,APLD 11/21/2025 22.00 P,-2,-113.68,-59.32,173.00,2152.8
2,97,ARE 11/21/2025 55.00 P,-2,-303.68,-277.36,581.04,1790.6
3,106,BAH 11/21/2025 80.00 P,-1,-70.34,-7.16,77.50,1444.7
4,112,BIRK 11/21/2025 37.50 P,-2,-64.67,24.67,40.00,1117.6
5,113,BLSH 11/21/2025 40.00 P,-2,-118.68,-631.34,750.02,2775.6
6,119,BRZE 11/21/2025 25.00 P,-2,-117.68,79.50,38.18,5000.0
7,129,CBRL 11/21/2025 32.50 P,-4,-478.36,-1041.40,1519.76,2873.2
8,166,DKS 11/21/2025 175.00 P,-1,-64.34,15.38,48.96,1798.9
9,177,EFX 11/21/2025 190.00 P,-1,-56.34,-29.55,85.89,2513.8


In [7]:
Total margin 117241
Weekly DataFrame:
            Cost Basis  Quantity  Premium   PnL  Margin
Date                                                   
2025-11-14        1580        28     1581  1197   10545
2025-11-21        7420       111     7421  3654   24961
2025-11-28        2229        38     2230  1350   11518
2025-12-05        2881        49     2882   192   15560
2025-12-12        2894        44     2895  -220   13875
2025-12-19        5371        89     5371  -945   29778
2025-12-26         306         5      307   -85    2047


Monthly Aggregated DataFrame:
         Cost Basis  Quantity  Premium   PnL  Margin
Month                                               
2025-11       11229       177    11232  6201   47024
2025-12       11452       187    11455 -1058   61260

SyntaxError: invalid syntax (3361696213.py, line 1)

3588

